In [4]:
!pip install openpyxl #open excel
!pip install biopython #read fasta file
!pip install propy3 #aac and dc

import numpy as np
import pandas as pd
import re

from Bio import SeqIO
import propy.AAComposition
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 KB 4.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.5/290.5 KB 5.9 MB/s eta 0:00:0000:01


/opt/conda/lib/python3.7/site-packages/propy/__init__.py:29: UserWarning: Python 3.6 and Python 3.7 might get deprecated. Please participate in the discussion: https://github.com/MartinThoma/propy3/issues/12
  "Python 3.6 and Python 3.7 might get deprecated. "


In [5]:
!pip3 install numpy

# Classifier Dataset

In [ ]:
original_df = pd.read_excel('../input/surface-protein-datasets/ML Final Dataset1.xlsx')

In [ ]:
df = original_df
df

In [ ]:
# Printing no.of null values in each column
print('Null value count:')
for col in df.columns:
  print(col,": ",df[col].isnull().sum())
print()

# Printing no.of unique values in each column
print('Unique value count:')
for col in df.columns:
  print(col,": ",df[col].value_counts().size)

# Dropping duplicate values
entry = df.Entry
dupli_vals = df[entry.isin(entry[entry.duplicated()])].sort_values("Entry").index
df.drop(dupli_vals, axis=0, inplace=True)

# Converting mass values from string to integer
df.Mass = [int(re.sub(",", "", x)) for x in df.Mass]

# Sequence Dataset

In [ ]:
fasta_paths = ['../input/surface-protein-datasets/uniprot-lpxtg.fasta',
               '../input/surface-protein-datasets/Non_surface.fasta']
seq_id_list, seq_list = [], []

# Extracting Entry ID and Sequence as a string from fasta files
for path in fasta_paths:
    for seq_record in SeqIO.parse(path, "fasta"):
        seq_id = seq_record.id
        idx = [i for i in range(len(seq_id)) if seq_id.startswith('|', i)]
        seq_id = seq_id[idx[0]+1:idx[1]]
        seq_id_list.append(seq_id)
        seq_list.append(str(seq_record.seq))

# Creating sequence dataframe
seq_dict = {'Entry':seq_id_list, 'Sequence':seq_list}
seq_df = pd.DataFrame(seq_dict)

In [ ]:
seq_df

In [ ]:
# Printing no.of null values in each column
print('Null value count:')
for col in seq_df.columns:
  print(col,": ",seq_df[col].isnull().sum())
print()
# Printing no.of unique values in each column
print('Unique value count:')
for col in seq_df.columns:
  print(col,": ",seq_df[col].value_counts().size)

# Dropping duplicate values
entry = seq_df.Entry
dupli_entry_idx = seq_df[entry.isin(entry[entry.duplicated()])].sort_values("Entry").index
seq_df.drop(dupli_entry_idx, axis=0, inplace=True)

# Dropping sequences containing 'B', 'U', and 'X'.
bux_index_list = []
bux = {'B','U','X'}
for s in seq_df.Sequence:
    if (bool(set(s)&(bux))):
        bux_index_list.append(seq_df.index[seq_df.Sequence==s][0])
seq_df.drop(bux_index_list, inplace=True, errors='ignore')

In [ ]:
# Merging the two dataframes and dropping columns with null values
# Also reset index for further data manipulation
final_df = df.merge(seq_df,how='outer')
final_df.dropna(inplace=True)
final_df.reset_index(inplace=True)
print(final_df.shape)

# Getting AAC and DC using propy

In [ ]:
# Getting amino acid composition
listx = [propy.AAComposition.CalculateAAComposition(seq) for seq in final_df.Sequence]
aac_df = pd.DataFrame.from_dict(listx)

# Getting dipeptide composition
listy = [propy.AAComposition.CalculateDipeptideComposition(seq) for seq in final_df.Sequence]
dc_df = pd.DataFrame.from_dict(listy)

# Getting final dataframe with 423 columns - 20-D AAC, 400-D DC, Mass, Length and Classifier
sp_df = aac_df.join(dc_df).join(final_df[['Mass','Length','Classifier']])
sp_df

# Combined Classifier

In [ ]:
seqs = sp_df[sp_df.columns[0:-1]]
classifier = sp_df['Classifier']

# Split the data into 80% training and 20% testing
X = seqs
y = classifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Create and train the Random Forest Classifier
RF_clf_main = RandomForestClassifier(n_estimators=100).fit(X_train, y_train)
y_pred = RF_clf_main.predict(X_test)

# Print accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print()

# 5-Fold cross-validation
kf = KFold(n_splits=5, shuffle=True)
score=cross_val_score(RF_clf_main,X,y,cv=kf)
print("Cross Validation Scores are {}".format(score))
print("Average Cross Validation score :{}".format(score.mean()))
print()

# Printing confusion matrix for the classifier
cm = confusion_matrix(y_test, y_pred)
print(cm)
print()

# Calculated statistics
tp, tn, fp, fn = cm[0,0],cm[1,1],cm[0,1],cm[1,0]
sn = tp/(tp+fn)
sp = tn/(tn+fp)
acc = (tp+tn)/(tp+tn+fp+fn)
mcc = ((tp*tn)-(fp*fn))/((tp+fn)*(tn+fn)*(tp+fp)*(tn+fp))
print('Calculated statistics:-')
print('Sensitivity, Sn = ',sn)
print('Specificity, Sp = ',sp)
print('Accuracy, ACC = ',acc)
print('Matthew’s correlation coefficient, MCC = ',mcc)

#### Confusion Matrix
![CONFUSION MATRIX](https://cdn.analyticsvidhya.com/wp-content/uploads/2021/05/confusion-matrix-1.png)

# Classification for Individual Features

## 1. AAC

In [ ]:
seqs = sp_df[sp_df.columns[0:20]]
classifier = sp_df['Classifier']

# Split the data into 80% training and 20% testing
X = seqs
y = classifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Create and train the Random Forest Classifier
RF_clf = RandomForestClassifier(n_estimators=100).fit(X_train, y_train)
y_pred = RF_clf.predict(X_test)

# Print accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print()

# Printing confusion matrix for the classifier
cm = confusion_matrix(y_test, y_pred)
print(cm)
print()

# Calculated statistics
tp, tn, fp, fn = cm[0,0],cm[1,1],cm[0,1],cm[1,0]
sn = tp/(tp+fn)
sp = tn/(tn+fp)
acc = (tp+tn)/(tp+tn+fp+fn)
mcc = ((tp*tn)-(fp*fn))/((tp+fn)*(tn+fn)*(tp+fp)*(tn+fp))
print('Calculated statistics:-')
print('Sensitivity, Sn = ',sn)
print('Specificity, Sp = ',sp)
print('Accuracy, ACC = ',acc)
print('Matthew’s correlation coefficient, MCC = ',mcc)

## 2. DC

In [ ]:
seqs = sp_df[sp_df.columns[20:420]]
classifier = sp_df['Classifier']

# Split the data into 80% training and 20% testing
X = seqs
y = classifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Create and train the Random Forest Classifier
RF_clf = RandomForestClassifier(n_estimators=100).fit(X_train, y_train)
y_pred = RF_clf.predict(X_test)

# Print accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print()

# Printing confusion matrix for the classifier
cm = confusion_matrix(y_test, y_pred)
print(cm)
print()

# Calculated statistics
tp, tn, fp, fn = cm[0,0],cm[1,1],cm[0,1],cm[1,0]
sn = tp/(tp+fn)
sp = tn/(tn+fp)
acc = (tp+tn)/(tp+tn+fp+fn)
mcc = ((tp*tn)-(fp*fn))/((tp+fn)*(tn+fn)*(tp+fp)*(tn+fp))
print('Calculated statistics:-')
print('Sensitivity, Sn = ',sn)
print('Specificity, Sp = ',sp)
print('Accuracy, ACC = ',acc)
print('Matthew’s correlation coefficient, MCC = ',mcc)

## 3. Mass

In [ ]:
seqs = sp_df[[sp_df.columns[420]]]
classifier = sp_df['Classifier']

# Split the data into 80% training and 20% testing
X = seqs
y = classifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Create and train the Random Forest Classifier
RF_clf = RandomForestClassifier(n_estimators=100).fit(X_train, y_train)
y_pred = RF_clf.predict(X_test)

# Print accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print()

# Printing confusion matrix for the classifier
cm = confusion_matrix(y_test, y_pred)
print(cm)
print()

# Calculated statistics
tp, tn, fp, fn = cm[0,0],cm[1,1],cm[0,1],cm[1,0]
sn = tp/(tp+fn)
sp = tn/(tn+fp)
acc = (tp+tn)/(tp+tn+fp+fn)
mcc = ((tp*tn)-(fp*fn))/((tp+fn)*(tn+fn)*(tp+fp)*(tn+fp))
print('Calculated statistics:-')
print('Sensitivity, Sn = ',sn)
print('Specificity, Sp = ',sp)
print('Accuracy, ACC = ',acc)
print('Matthew’s correlation coefficient, MCC = ',mcc)

## 4. Length

In [ ]:
seqs = sp_df[[sp_df.columns[421]]]
classifier = sp_df['Classifier']

# Split the data into 80% training and 20% testing
X = seqs
y = classifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Create and train the Random Forest Classifier
RF_clf = RandomForestClassifier(n_estimators=100).fit(X_train, y_train)
y_pred = RF_clf.predict(X_test)

# Print accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print()

# Printing confusion matrix for the classifier
cm = confusion_matrix(y_test, y_pred)
print(cm)
print()

# Calculated statistics
tp, tn, fp, fn = cm[0,0],cm[1,1],cm[0,1],cm[1,0]
sn = tp/(tp+fn)
sp = tn/(tn+fp)
acc = (tp+tn)/(tp+tn+fp+fn)
mcc = ((tp*tn)-(fp*fn))/((tp+fn)*(tn+fn)*(tp+fp)*(tn+fp))
print('Calculated statistics:-')
print('Sensitivity, Sn = ',sn)
print('Specificity, Sp = ',sp)
print('Accuracy, ACC = ',acc)
print('Matthew’s correlation coefficient, MCC = ',mcc)

# Classifying a new entry

In [ ]:
entry = {'Sequence':'MKALLLKTSVWLVLLFSVMGLWQVSNAAEQYTPIKAHVVTTIDKATTDKQQVTPTKEAAHQFGEEAATNVSASAQGTADEINNKVTSNAFSNKPSTAVSTKVNETHDVDTQQASTQKPTQSATFTLSNAKTASLSPRMFAANVPQTTTHKILHTNDIHGRLAEEKGRVIGMAKLKTIKEQEKPDLMLDAGDAFQGLPLSNQSKGEEMAKAMNAVGYDAMAVGNHEFDFGYDQLKKLEGMLDFPMLSTNVYKDGKRAFKPSTIVTKNGIRYGIIGVTTPETKTKTRPEGIKGVEFRDPLQSVTAEMMRIYKDVDTFVVISHLGIDPSTQETWRGDYLVKQLSQNPQLKKRITVIDGHSHTVLQNGQIYNNDALAQTGTALANIGKVTFNYRNGEVSNIKPSLINVKDVENVTPNKALAEQINQADQTFRAQTAEVIIPNNTIDFKGERDDVRTRETNLGNAIADAMEAYGVKNFSKKTDFAVTNGGGIRASIAKGKVTRYDLISVLPFGNTIAQIDVKGSDVWTAFEHSLGAPTTQKDGKTVLTANGGLLHISDSIRVYYDMNKPSGKRINAIQILNKETGKFENIDLKRVYHVTMNDFTASGGDGYSMFGGPREEGISLDQVLASYLKTANIAKYDTTEPQRMLLGKPAVSEQPAKGQQGSKGSESGKDVQPIGDDKAMNPAKQPATGKVVLLPTHRGTVSSGTEGSGRTLEGATVSSKSGNQLVRMSVPKGSAHEKQLPKTGTNQSSSPAAMFVLVAGIGLIATVRRRKAS',
                     'Mass':83498, 'Length':772} #This is actually the first entry in the dataset
entry = pd.DataFrame(entry,index=[0])

# Getting amino acid composition
aac_df = pd.DataFrame([propy.AAComposition.CalculateAAComposition(ent) for ent in entry['Sequence']])

# Getting dipeptide composition
dc_df = pd.DataFrame([propy.AAComposition.CalculateDipeptideComposition(ent) for ent in entry['Sequence']])

# Merging the dataframes
entry_df = aac_df.join(dc_df).join(entry).drop(['Sequence'],axis=1)

# Classifying
prediction = RF_clf_main.predict(pd.DataFrame(entry_df))
print(prediction)

# Graphs

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sp_df[['Mass','Length','Classifier']]

In [ ]:
sns.lineplot(data=sp_df, x='Length', y='Mass', hue='Classifier');

In [ ]:
sns.displot(data = sp_df, x='Mass',hue='Classifier', kind='kde');

In [ ]:

sns.barplot(x='Classifier', y='Mass', data=sp_df);

In [ ]:
sp_df.Mass.describe()